# 3 start auth pods on Azure AKS

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/core
```

## load environment variables

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## start authorization & authentication service on AKS

### create `secrets/auth-tokens.json`

example)
```json
[
  {
    "host": "api\\..+$",
    "settings": {
      "bearer_tokens": [
        {
          "token": "nrWtb8sS0MmwlkhHXv0DC6orPMpFFbni",
          "allowed_paths": ["^/orion/.*$", "^/idas/.*$", "^/comet/.*$"]
        }
      ],
      "basic_auths": [],
      "no_auths": {}
    }
  },
  {
    "host": "kibana\\..+$",
    "settings": {
      "bearer_tokens": [],
      "basic_auths": [
        {
          "username": "yW7FvSGD",
          "password": "6BoTFE5xfUlX3ssV",
          "allowed_paths": ["^.*$"]
        }
      ],
      "no_auths": {
        "allowed_paths": []
      }
    }
  },
  {
    "host": "grafana\\..+$",
    "settings": {
      "bearer_tokens": [],
      "basic_auths": [],
      "no_auths": {
        "allowed_paths": ["^.*$"]
      }
    }
  }
]
```

#### for macOS

In [ ]:
cat << __EOS__ > secrets/auth-tokens.json
[
    {
        "host": "api\\\\..+$",
        "settings": {
            "bearer_tokens": [
                {
                    "token": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 32)",
                    "allowed_paths": ["^/orion/.*$", "^/idas/.*$", "^/comet/.*$"]
                }
            ],
            "basic_auths": [],
            "no_auths": {
                "allowed_paths": []
            }
        }
    }, {
        "host": "kibana\\\\..+$",
        "settings": {
            "bearer_tokens": [],
            "basic_auths": [
                {
                    "username": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 8)",
                    "password": "$(cat /dev/urandom | LC_CTYPE=C tr -dc 'a-zA-Z0-9' | head -c 16)",
                    "allowed_paths": ["^.*$"]
                }
            ],
            "no_auths": {
                "allowed_paths": []
            }
        }
    }, {
        "host": "grafana\\\\..+$",
        "settings": {
            "bearer_tokens": [],
            "basic_auths": [],
            "no_auths": {
                "allowed_paths": ["^.*$"]
            }
        }
    }
]
__EOS__

#### for Ubuntu

In [ ]:
cat << __EOS__ > secrets/auth-tokens.json
[
    {
        "host": "api\\\\..+$",
        "settings": {
            "bearer_tokens": [
                {
                    "token": "$(cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 32)",
                    "allowed_paths": ["^/orion/.*$", "^/idas/.*$", "^/comet/.*$"]
                }
            ],
            "basic_auths": [],
            "no_auths": {
                "allowed_paths": []
            }
        }
    }, {
        "host": "kibana\\\\..+$",
        "settings": {
            "bearer_tokens": [],
            "basic_auths": [
                {
                    "username": "$(cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 8)",
                    "password": "$(cat /dev/urandom 2>/dev/null | head -n 40 | tr -cd 'a-zA-Z0-9' | head -c 16)",
                    "allowed_paths": ["^.*$"]
                }
            ],
            "no_auths": {
                "allowed_paths": []
            }
        }
    }, {
        "host": "grafana\\\\..+$",
        "settings": {
            "bearer_tokens": [],
            "basic_auths": [],
            "no_auths": {
                "allowed_paths": ["^.*$"]
            }
        }
    }
]
__EOS__

### register auth-tokens to kubernetes secrets

In [ ]:
kubectl create secret generic auth-tokens --from-file=./secrets/auth-tokens.json

### start auth service

In [ ]:
kubectl apply -f auth/auth-service.yaml

In [ ]:
kubectl apply -f auth/auth-deployment.yaml

In [ ]:
kubectl get pods -l app=auth

example)
```
NAME                    READY   STATUS    RESTARTS   AGE
auth-67ff6bd94b-bkk96   1/1     Running   0          23s
auth-67ff6bd94b-bqstl   1/1     Running   0          23s
auth-67ff6bd94b-p2xzt   1/1     Running   0          23s
```

In [ ]:
kubectl get services -l app=auth

example)
```
NAME   TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
auth   ClusterIP   10.0.221.96   <none>        3000/TCP   43s
```